<b>Expectation Maximization for Gaussian Mixtures</b><br>
--
Created by : Adirahman (Suryo)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal as gauss
from sklearn.metrics.pairwise import euclidean_distances as l2

In [ ]:
faithful_df = pd.read_table('faithful.dat', sep='\s+', header=13, index_col=0)
display(faithful_df)

X = faithful_df.to_numpy()

,eruptions,waiting
1,3.600,79
2,1.800,54
3,3.333,74
4,2.283,62
5,4.533,85
...,...,...
268,4.117,81
269,2.150,46
270,4.417,90
271,1.817,46


In [ ]:
X

array([[ 3.6  , 79.   ],
       [ 1.8  , 54.   ],
       [ 3.333, 74.   ],
       [ 2.283, 62.   ],
       [ 4.533, 85.   ],
       [ 2.883, 55.   ],
       [ 4.7  , 88.   ],
       [ 3.6  , 85.   ],
       [ 1.95 , 51.   ],
       [ 4.35 , 85.   ],
       [ 1.833, 54.   ],
       [ 3.917, 84.   ],
       [ 4.2  , 78.   ],
       [ 1.75 , 47.   ],
       [ 4.7  , 83.   ],
       [ 2.167, 52.   ],
       [ 1.75 , 62.   ],
       [ 4.8  , 84.   ],
       [ 1.6  , 52.   ],
       [ 4.25 , 79.   ],
       [ 1.8  , 51.   ],
       [ 1.75 , 47.   ],
       [ 3.45 , 78.   ],
       [ 3.067, 69.   ],
       [ 4.533, 74.   ],
       [ 3.6  , 83.   ],
       [ 1.967, 55.   ],
       [ 4.083, 76.   ],
       [ 3.85 , 78.   ],
       [ 4.433, 79.   ],
       [ 4.3  , 73.   ],
       [ 4.467, 77.   ],
       [ 3.367, 66.   ],
       [ 4.033, 80.   ],
       [ 3.833, 74.   ],
       [ 2.017, 52.   ],
       [ 1.867, 48.   ],
       [ 4.833, 80.   ],
       [ 1.833, 59.   ],
       [ 4.783, 90.   ],


In [ ]:
# Class structure that used for plotting the data

class Plotter:
    def __init__(self, save_plot=False):
        self.img_idx = 1
        self.save_plot = save_plot
        
    def plot_clusters(self, X, mu_pred=None, labels=None, sigma_pred=None):
        plt.figure(figsize=(6, 6))
        x_min, x_max = 1.1, 5.5
        y_min, y_max = 40, 100
        
        if mu_pred is not None and sigma_pred is not None:
            x1 = np.linspace(x_min, x_max, 100)
            x2 = np.linspace(y_min, y_max, 100)
            xx1, xx2 = np.meshgrid(x1, x2)
            xx12 = np.hstack((xx1.reshape(-1, 1), xx2.reshape(-1, 1)))
            cmaps = ['Blues', 'Oranges', 'Greens', 'Reds']
            
            for d in range(mu_pred.shape[0]):
                plt.contourf(x1, 
                             x2, 
                             gauss.pdf(xx12, mean=mu_pred[d], cov=sigma_pred[d]).reshape(100, 100), 
                             alpha=0.2, 
                             levels=100,
                             cmap=cmaps[d%mu_pred.shape[0]])
        
        if mu_pred is None or labels is None:
            plt.scatter(X[:, 0], X[:, 1], c='grey', s=10)
        else:
            for d in range(mu_pred.shape[0]):
                plt.scatter(X[labels==d, 0], X[labels==d, 1], label='Cluster %d' % (d+1), s=10)
            plt.scatter(mu_pred[:, 0], mu_pred[:, 1], label='Cluster centers', c='k', s=50)
            plt.legend()
    
        plt.xlabel('Eruption time [min]')
        plt.xlim([x_min, x_max])
        
        plt.ylabel('Waiting time to next eruption [min]')
        plt.ylim([y_min, y_max])
    
        plt.title('Old Faithful Geyser Data')

        if self.save_plot:
            plt.savefig('out/EM_%03d.png' % self.img_idx)
            self.img_idx += 1
        else:
            plt.show()
        
        plt.close()

In [ ]:
plotter = Plotter(save_plot=True)
plotter.plot_clusters(X)

In [ ]:
def kmeans_cluster(X, K, eps=1e-6, max_iter=100):
    N, D = X.shape
    mu_pred = X[np.random.choice(N, size=K, replace=False), :]
    
    obj = 1e9
    prev_obj = None
    for _ in range(max_iter):
        distances = l2(X, mu_pred, squared=True)
        labels = np.argmin(distances, axis=1)
        plotter.plot_clusters(X, mu_pred=mu_pred, labels=labels)
        
        prev_obj = obj
        obj = 0
        for i in range(N):
            obj += distances[i, labels[i]]
            
        if prev_obj - obj < eps:
            break
            
        for i in range(K):
            if np.sum(labels==i) == 0:
                return kmeans_cluster(X, K)
            
            mu_pred[i, :] = np.sum(X[labels==i,:], axis=0)/np.sum(labels==i)
    
    return obj, labels, mu_pred

In [ ]:
K = 2
_, _, mu_pred = kmeans_cluster(X, K)

In [ ]:
def em_gaussian_mixture(X, K, mu_pred, eps=1e-3, max_iter=100):
    N, D = X.shape
    pi_pred = np.ones(shape=(K), dtype=np.float)/K
    sigma_pred = np.zeros(shape=(K, D, D), dtype=np.float)
    for d in range(D):
        sigma_pred[:, d, d] = np.var(X[:, d])/10
    
    obj = 1e9
    prev_obj = None
    for _ in range(max_iter):
        gamma = np.zeros(shape=(N, K))
        for k in range(K):
            gamma[:, k] = gauss.pdf(X, mean=mu_pred[k], cov=sigma_pred[k]) * pi_pred[k]
        gamma = gamma/np.sum(gamma, axis=1, keepdims=True)
        prev_obj = obj
        obj = 0
        for k in range(K):
            obj += np.sum(gamma[:, k] * (np.log(gauss.pdf(X, mean=mu_pred[k], cov=sigma_pred[k])) +\
                                         np.log(pi_pred[k])))
            
        labels = np.argmax(gamma, axis=1)
        plotter.plot_clusters(X, mu_pred=mu_pred, labels=labels, sigma_pred=sigma_pred)
        if np.abs(obj - prev_obj) < eps:
            break
            
        pi_pred = np.sum(gamma, axis=0)/np.sum(gamma)
        for k in range(K):
            mu_pred[k, :] = np.sum(gamma[:, k].reshape(-1,1) * X, axis=0) /\
                            np.sum(gamma[:, k], keepdims=True)
            sigma_pred[k, :, :] = np.sum(gamma[:, k].reshape(-1, 1, 1) *\
                                         ((X-mu_pred[k, :]).reshape(N, D, 1) @\
                                          (X-mu_pred[k, :]).reshape(N, 1, D)), axis=0) /\
                                  np.sum(gamma[:, k])
    return obj, labels, pi_pred, mu_pred, sigma_pred

In [ ]:
obj, _, pi_pred, mu_pred, sigma_pred = em_gaussian_mixture(X, K, mu_pred)

In [ ]:
obj

-1130.9586076229793

In [ ]:
print('Expected Complete-Data Log-Likelihood:')
print('--------------------------------------')
print('%.3f' % obj)
print()
print('Cluster probabilities:')
print('----------------------')
print(pi_pred)
print()
print('Cluster means:')
print('--------------')
print(mu_pred)
print()
print('Cluster covariance matrices:')
print('----------------------------')
print(sigma_pred)

Expected Complete-Data Log-Likelihood:
--------------------------------------
-1130.959

Cluster probabilities:
----------------------
[0.35587398 0.64412602]

Cluster means:
--------------
[[ 2.03639118 54.47854382]
 [ 4.28966439 79.96814438]]

Cluster covariance matrices:
----------------------------
[[[ 0.06916984  0.43519023]
  [ 0.43519023 33.69743623]]

 [[ 0.16996537  0.94057034]
  [ 0.94057034 36.04577243]]]
